In [ ]:
# Laboratorio de Datos
# Trabajo Práctico Nª 1 
# Integrantes: Román Parisi y Joaquín Núñez

In [64]:
# Importamos las librerías necesarias para el desarrollo del TP:

import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import datetime # Para poder convertir al tipo datetime en el punto 1a
import matplotlib.pyplot as plt

from sklearn import linear_model    
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from formulaic import model_matrix
from formulaic import Formula

In [8]:
# Accedemos al dataset y lo observamos

datos = pd.read_csv("sube-2023.csv")
datos.head()

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO


In [9]:
# Primer Ejercicio: PROCESAMIENTO DE DATOS 

In [65]:
# 1.(a)

datos.dtypes # Visualizamos el tipo de datos de cada columna ("DIA_TRANSPORTE" es de tipo object)
datos["DIA_TRANSPORTE"] = pd.to_datetime(datos["DIA_TRANSPORTE"]) # Transformamos la columna "DIA_TRANSPORTE" al tipo datetime
datos.dtypes # Volvemos a ver el tipo de datos de cada columna y ahora "DIA_TRANSPORTE" es tipo datetime

DIA_TRANSPORTE     datetime64[ns]
NOMBRE_EMPRESA             object
LINEA                      object
AMBA                       object
TIPO_TRANSPORTE            object
JURISDICCION               object
PROVINCIA                  object
MUNICIPIO                  object
CANTIDAD                    int64
DATO_PRELIMINAR            object
FECHA_DIA                  object
FECHA_ORDINAL               int64
FECHA_MES                  object
dtype: object

In [66]:
# 1.(b)

def dia_de_la_semana(fecha: datetime) -> str:
    return fecha.strftime("%a") # Funcón para devolver el nombre del día de la semana dado un elemento del tipo datetime 

def fecha_ordinal(fecha: datetime) -> int:
    return int(fecha.strftime("%d")) # Funcón para devolver el cardinal de la fecha dado un elemento del tipo datetime 

def mes_del_año(fecha: datetime) -> str:
    return fecha.strftime("%m") # Funcón para devolver el número del mes dado un elemento del tipo datetime 

# Generamos nuevas columnas y les asignamos los datos pedidos utilizando las funciones de arriba

datos["FECHA_DIA"] = datos["DIA_TRANSPORTE"].apply(dia_de_la_semana) 
datos["FECHA_ORDINAL"] = datos["DIA_TRANSPORTE"].apply(fecha_ordinal)
datos["FECHA_MES"] = datos["DIA_TRANSPORTE"].apply(mes_del_año)


In [67]:
# 2 

datos_amba = datos[datos["AMBA"] == "SI"]
columnas_a_eliminar = ["NOMBRE_EMPRESA", "AMBA", "MUNICIPIO", "DATO_PRELIMINAR", "PROVINCIA", "NOMBRE_EMPRESA"]
datos_amba.drop(columnas_a_eliminar, axis = 1, inplace = True)
nombre_de_columnas = ["fecha", "fecha_dia", "fecha_mes", "fecha_ordinal", "jurisdiccion", "linea", "pasajeros", "tipo_transporte"]
datos_amba = datos_amba.rename(columns = {"DIA_TRANSPORTE" : "fecha", "LINEA" : "linea", "TIPO_TRANSPORTE" : "tipo_transporte", "JURISDICCION" : "jurisdiccion", "CANTIDAD" : "pasajeros", "FECHA_DIA" : "fecha_dia", "FECHA_ORDINAL" : "fecha_ordinal", "FECHA_MES" : "fecha_mes"})
datos_amba = datos_amba.reindex(columns = nombre_de_columnas)
datos_amba.head()

,fecha,fecha_dia,fecha_mes,fecha_ordinal,jurisdiccion,linea,pasajeros,tipo_transporte
0,2023-01-01,dom,01,1,MUNICIPAL,1,61,COLECTIVO
1,2023-01-01,dom,01,1,MUNICIPAL,2B,11,COLECTIVO
2,2023-01-01,dom,01,1,MUNICIPAL,BS_AS_LINEA 715M,1707,COLECTIVO
3,2023-01-01,dom,01,1,PROVINCIAL,BS_AS_LINEA_326,438,COLECTIVO
4,2023-01-01,dom,01,1,MUNICIPAL,BS_AS_LINEA_512,1189,COLECTIVO
